# Functional API **Project**

Based on **Patric Loeber** video: https://www.youtube.com/watch?v=LwM7xabuiIc&t=6604s

The functional API  is another way to define our models. It is more flexible than the beginner friendly sequential API.

### Introduction

https://www.tensorflow.org/guide/keras/functional

The Keras functional API is a way to create models that are more flexible than the tf.keras.Sequential API. The functional API can handle models with non-linear topology, shared layers, and even multiple inputs or outputs.

The main idea is that a deep learning model is usually a directed acyclic graph (DAG) of layers. So the functional API is a way to build graphs of layers.

## Code

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

### Typical example of the sequential API

With the Sequential API we can only have a linear graph:

a
|
b
|
c

one input and then one layer after the other and only one output.

In [3]:
# model; Sequential: one input, one output
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28,28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10)
])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 128)               100480    
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101770 (397.54 KB)
Trainable params: 101770 (397.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


### Functional API

With the functional API we have a lot of more options like this below. For example we can have multiple inputs, outputs or even somwhere in the middle we can have different paths.

```python
#a           a           a   b           a
#|           |            \ /           / \
#b           b             c           b   c
#|          / \            |            \ /
#c         c   d           d             d
```

In [5]:
# We start by creating an input layer

inputs = keras.Input(shape=(28,28))

# Then we define our layers
flatten = keras.layers.Flatten()
dense1 = keras.layers.Dense(128, activation='relu')
dense2 = keras.layers.Dense(10)

# Calling layers, all layers above are callable like functions

x = flatten(inputs)
x = dense1(x)

outputs = dense2(x)

# Creating model
model = keras.Model(inputs = inputs, outputs = outputs, name='functionam_model')

print(model.summary())

Model: "functionam_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 128)               100480    
                                                                 
 dense_5 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101770 (397.54 KB)
Trainable params: 101770 (397.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


### Functional API with multiple outputs

In [7]:
# We start by creating an input layer

inputs = keras.Input(shape=(28,28))

# Then we define our layers
flatten = keras.layers.Flatten()
dense1 = keras.layers.Dense(128, activation='relu')
dense2 = keras.layers.Dense(10)
dense2_2 = keras.layers.Dense(1) # second binary classification task

# Calling layers, all layers above are callable like functions

x = flatten(inputs)
x = dense1(x)

outputs = dense2(x)
outputs2 = dense2_2(x)

# Creating model, in outputs we use a list to have multiple outputs -> we can do same with multiple inputs etc.
model = keras.Model(inputs = inputs, outputs = [outputs, outputs2], name='functionam_model')

print(model.summary())

Model: "functionam_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 28, 28)]             0         []                            
                                                                                                  
 flatten_4 (Flatten)         (None, 784)                  0         ['input_4[0][0]']             
                                                                                                  
 dense_9 (Dense)             (None, 128)                  100480    ['flatten_4[0][0]']           
                                                                                                  
 dense_10 (Dense)            (None, 10)                   1290      ['dense_9[0][0]']             
                                                                                   

### Converting to a sequential model from a functional model

It is very simple because we can iterate over the layers. this operation only works if the layer graph is linear.

In [ ]:
# Define new empty sequential model

new_model = keras.models.Sequential()

for layer in model.layers:
    new_model.add(layer)

### Converting from to a functional model from a sequential model

For a funcional API we have to define an input and then we can access the first layer by calling model layers 0 and then pass in the inputs. Then we do a for loop and iterate over all the other layers starting at 1. Then we call the layer and give it the input and assign it. At the end we say our output is the output from  the last iteration and then we define model.

It is very flexible because we can leav out certain layers by doing if statment or we can use slicing operators, it is very common that we leaver out the last layer.

this is very powerful for transfer learning which is very important concept.

In [ ]:
inputs = keras.Input(shape=(28,28))
x = new_model.layers[0](inputs)
for layer in new_model.layers[1:-1]:
    x = layer(x)
outputs = x

model = keras.Model(inputs = inputs, outputs = outputs, name='functional_model_from_seq')

### Advantages of the functional API

+ Models with multiple inputs and outputs
+ Shared layers
+ Extract and reuse nodes in the graph of layers
+ Model are callable like layers (put model into sequential)

### Accessing the inputs and the outputs

For every model we can acces the inputs and the outputs of a model and the layer.

In [9]:
inputs = model.inputs
outputs = model.outputs

print(inputs)
print(outputs)

[<KerasTensor: shape=(None, 28, 28) dtype=float32 (created by layer 'input_4')>]
[<KerasTensor: shape=(None, 10) dtype=float32 (created by layer 'dense_10')>, <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'dense_11')>]


We can also get the input and the output for each layer.

In [10]:
input0 = model.layers[0].input
output0 = model.layers[0].output

print(input0)
print(output0)

KerasTensor(type_spec=TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='input_4'), name='input_4', description="created by layer 'input_4'")
KerasTensor(type_spec=TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='input_4'), name='input_4', description="created by layer 'input_4'")


### Usage of accessning the inputs and the outputs in transfer learning

Let's say that we have already trained model and we want to modify onbly the last layer. So we want to cut out the last layer so we want to cut out the last layer of the base model and use our own. We can do this by accessing the output of the second last last layer and then we have to define a new output and pass inthis last output. Then we set up a new functional model.

In [ ]:
base_model = keras.applications.VGG16()

x = base_model.layers[-2].output
new_outputs = keras.layers.Dense(1)(x)

new_model = keras.Model(inputs=base_model.inputs, outputs=new_outputs)

## Example with a classification task

We have two outputs that we want to predict. In this case we use MNIST dataset so we want to predict the hand written digit. But now we also assume that we want to predict if the digit was written with a right hand or a left hand.

In [11]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [15]:
# Define Functional Model
inputs = keras.Input(shape=(28,28))
flatten = keras.layers.Flatten()
dense1 = keras.layers.Dense(128, activation='relu')

dense2 = keras.layers.Dense(10, activation='softmax', name="category_output")
dense3 = keras.layers.Dense(1, activation='sigmoid', name="leftright_output")

In [16]:
x = flatten(inputs)
x = dense1(x)

outputs1 = dense2(x)
outputs2 = dense3(x)

model = keras.Model(inputs=inputs, outputs=[outputs1, outputs2], name="mnist_model")

In [17]:
model.summary()

Model: "mnist_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 28, 28)]             0         []                            
                                                                                                  
 flatten_7 (Flatten)         (None, 784)                  0         ['input_7[0][0]']             
                                                                                                  
 dense_14 (Dense)            (None, 128)                  100480    ['flatten_7[0][0]']           
                                                                                                  
 category_output (Dense)     (None, 10)                   1290      ['dense_14[0][0]']            
                                                                                        

In [20]:
# loss and optimizer
loss1 = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
loss2 = keras.losses.BinaryCrossentropy(from_logits=False)

optim = keras.optimizers.legacy.Adam(learning_rate=0.001)

metrics = ["accuracy"]

losses = {
    "category_output": loss1,
    "leftright_output": loss2,
}

model.compile(loss=losses, optimizer=optim, metrics=metrics)

In [21]:
# create data with 2 labels
mnist = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# 0=left, 1=right -> dummy example because mnist doesn't have information about which hand was used to write the numbers
y_leftright = np.zeros(y_train.shape, dtype=np.uint8)
for idx, y in enumerate(y_train):
    if y > 5:
        y_leftright[idx] = 1

print(y_train.dtype, y_train[0:20])
print(y_leftright.dtype, y_leftright[0:20])

y = {"category_output": y_train,
    "leftright_output": y_leftright}

uint8 [5 0 4 1 9 2 1 3 1 4 3 5 3 6 1 7 2 8 6 9]
uint8 [0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1 1 1]


In [22]:
# training
model.fit(x_train, y=y, epochs=5,
          batch_size=64, verbose=2)

Epoch 1/5
938/938 - 1s - loss: 0.4993 - category_output_loss: 0.3102 - leftright_output_loss: 0.1890 - category_output_accuracy: 0.9128 - leftright_output_accuracy: 0.9313 - 807ms/epoch - 860us/step
Epoch 2/5
938/938 - 1s - loss: 0.2403 - category_output_loss: 0.1431 - leftright_output_loss: 0.0972 - category_output_accuracy: 0.9580 - leftright_output_accuracy: 0.9670 - 584ms/epoch - 622us/step
Epoch 3/5
938/938 - 1s - loss: 0.1787 - category_output_loss: 0.1020 - leftright_output_loss: 0.0767 - category_output_accuracy: 0.9704 - leftright_output_accuracy: 0.9741 - 577ms/epoch - 615us/step
Epoch 4/5
938/938 - 1s - loss: 0.1432 - category_output_loss: 0.0788 - leftright_output_loss: 0.0644 - category_output_accuracy: 0.9770 - leftright_output_accuracy: 0.9778 - 569ms/epoch - 606us/step
Epoch 5/5
938/938 - 1s - loss: 0.1169 - category_output_loss: 0.0622 - leftright_output_loss: 0.0547 - category_output_accuracy: 0.9820 - leftright_output_accuracy: 0.9817 - 566ms/epoch - 603us/step


In [23]:
# list with 2 predictions
predictions = model.predict(x_test)
len(predictions)

313/313 [==============================] - 0s 317us/step


2

In [24]:
prediction_category = predictions[0]
prediction_lr = predictions[1]

pr_cat = prediction_category[0:20]
pr_lr = prediction_lr[0:20]

labels_cat = np.argmax(pr_cat, axis=1)
labels_lr = np.array([1 if p >= 0.5 else 0 for p in pr_lr])

In [27]:
print(f"First 20 real category labels: {y_test[0:20]}")
print(f"First 20 predicted category labels: {labels_cat}")
print(f"First 20 predicted left(0) or right(1) hand labels: {labels_lr}")

First 20 real category labels: [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
First 20 predicted category labels: [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
First 20 predicted left(0) or right(1) hand labels: [1 0 0 0 0 0 0 1 0 1 0 1 1 0 0 0 1 1 0 0]
